In [1]:
import torch

In [2]:
t1 = torch.tensor(4.)
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [5]:
t2 = torch.tensor([1., 2, 3, 4])
t2

tensor([1., 2., 3., 4.])

In [6]:
t3 = torch.tensor([[5., 6],
                   [7, 8],
                   [9, 10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [7]:
t4 = torch.tensor([
    [[11, 12, 13],
     [13, 14, 15]],
    [[15, 16, 17],
     [17, 18, 19.]]])
t4


tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [17., 18., 19.]]])

In [8]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [9]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [10]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [11]:
print(t4)
t4.shape

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [17., 18., 19.]]])


torch.Size([2, 2, 3])

In [12]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [13]:
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

In [14]:
y.backward()

In [15]:
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


In [16]:
import numpy as np
import torch

inputs = np.array([[73, 67, 43],
                  [91, 88, 64],
                  [87, 134, 58],
                  [102, 43, 37],
                  [69, 96, 70]], dtype='float32')

In [17]:
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [18]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [19]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[-0.0247,  1.2456, -1.6673],
        [ 0.1216,  0.2639, -2.0648]], requires_grad=True)
tensor([0.0447, 0.6121], requires_grad=True)


In [20]:
def model(x):
    return x @ w.t() + b

In [21]:
preds = model(inputs)
print(preds)

tensor([[  10.0079,  -61.6146],
        [   0.7090,  -97.2441],
        [  68.1101,  -73.2033],
        [ -10.5983,  -52.0319],
        [   1.2127, -110.1975]], grad_fn=<AddBackward0>)


In [22]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [23]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()


In [24]:
loss = mse(preds, targets)
print(loss)

tensor(18217.6094, grad_fn=<DivBackward0>)


In [25]:
loss.backward()

In [26]:
print(w)
print(w.grad)

tensor([[-0.0247,  1.2456, -1.6673],
        [ 0.1216,  0.2639, -2.0648]], requires_grad=True)
tensor([[ -5087.9351,  -5627.9268,  -3679.8291],
        [-14096.7295, -15945.2461,  -9928.9697]])


In [27]:
w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [28]:
preds = model(inputs)
print(preds)


tensor([[  10.0079,  -61.6146],
        [   0.7090,  -97.2441],
        [  68.1101,  -73.2033],
        [ -10.5983,  -52.0319],
        [   1.2127, -110.1975]], grad_fn=<AddBackward0>)


In [29]:
loss = mse(preds, targets)
print(loss)

tensor(18217.6094, grad_fn=<DivBackward0>)


In [30]:
loss.backward()
print(w.grad)
print(b.grad)


tensor([[ -5087.9351,  -5627.9268,  -3679.8291],
        [-14096.7295, -15945.2461,  -9928.9697]])
tensor([ -62.3117, -170.8583])


In [31]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
    

In [32]:
print(w)
print(b)

tensor([[ 0.0262,  1.3019, -1.6305],
        [ 0.2626,  0.4233, -1.9655]], requires_grad=True)
tensor([0.0453, 0.6138], requires_grad=True)


In [33]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)


tensor(12547.9424, grad_fn=<DivBackward0>)


In [38]:
for i in range(500):
    preds = model(inputs)
    loss  = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
        

In [39]:
preds = model(inputs)
loss  = mse(preds, targets)
print(loss)

tensor(129.6229, grad_fn=<DivBackward0>)


In [40]:
preds

tensor([[ 58.0925,  71.4577],
        [ 74.5794,  92.2356],
        [134.5496, 150.2967],
        [ 25.5897,  42.1586],
        [ 86.0305, 101.5857]], grad_fn=<AddBackward0>)

In [41]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [42]:
import torch.nn as nn

In [43]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43],
                   [91, 88, 64], [87, 134, 58], [102, 43, 37],
                   [69, 96, 70], [73, 67, 43], [91, 88, 64],
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]],
                 dtype='float32')

targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

In [44]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [45]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [46]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [47]:
from torch.utils.data import TensorDataset

In [49]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]


(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [50]:
from torch.utils.data import DataLoader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
